In [1]:
import os
import re
import time
import shutil
import pyodbc
import fnmatch
import numpy as np
import pandas as pd
import win32com.client
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

แบ่ง iri ออกเป็น 4 ช่วง (20 -> 5) เพื่อแปลงค่า iri

In [2]:
def generate_parts(target_values, num_parts, tolerance):
    parts_list = []
    for target_value in target_values:
        total_sum = target_value * num_parts
        while True:
            # Generate random parts
            parts = np.random.uniform(low=total_sum / num_parts * 0.9, high=total_sum / num_parts * 1.1, size=num_parts)
            # Ensure the sum is correct
            if np.abs(np.sum(parts) - total_sum) < tolerance:
                parts_list.append(parts)
                break
    return parts_list

เพิ่ม chainage event_str event_end ให้ iri/rut return เป็น dic 
EX.iri_dataframes['xw_iri_20240726RUN03.csv'] rutting_dataframes['xw_rutting_20240726RUN03.csv']

In [1]:
# Find all relevant CSV files and process them
def process_csv_files(path):
    iri_dataframes = {}
    rutting_dataframes = {}

    for root, dirs, files in os.walk(path):
        # Find files
        iri_files = [f for f in files if f.endswith('.csv') and 'xw_iri_qgis' in f]
        rutting_files = [f for f in files if f.endswith('.csv') and 'xw_rutting' in f]
        
        # Process 'xw_iri_qgis' files
        for filename in iri_files:
            file_path = os.path.join(root, filename)
            iri_df = pd.read_csv(file_path, delimiter=';')
            iri_df.columns = iri_df.columns.str.strip()
            survey_code = filename.split('_')[3].split('.')[0]
            iri_df['survey_code'] = survey_code
            iri_df['iri'] = (iri_df['iri left (m/km)'] + iri_df['iri right (m/km)']) / 2        
            iri_df.drop(columns=['geometry'], errors='ignore', inplace=True)
            
            # Generate random values
            target_values = iri_df['iri']
            num_parts = 4
            tolerance = 0.3
            parts_list = generate_parts(target_values, num_parts, tolerance)

            # Expand DataFrame by repeating the rows
            iri_df = iri_df.loc[iri_df.index.repeat(num_parts)].reset_index(drop=True)
            iri_df['iri_lane'] = np.concatenate(parts_list)
            
            # Set initial event columns
            increment = 5 if fnmatch.fnmatch(filename, '*xw_iri_qgis*') else 5
            iri_df['iri_chainage'] = iri_df.index * 5
            iri_df['event_start'] = range(0, len(iri_df) * increment, increment)
            iri_df['event_end'] = iri_df['event_start'] + increment
            iri_dataframes[filename] = iri_df
            
            # print(f"Updated {filename} into IRI DataFrame.")
        
        # Process 'xw_rutting' files
        for filename in rutting_files:
            file_path = os.path.join(root, filename)
            rut_df = pd.read_csv(file_path, delimiter=';')
            rut_df.columns = rut_df.columns.str.strip()
            if 'Unnamed: 5' in rut_df.columns:
                rut_df.drop(columns=['Unnamed: 5'], inplace=True, errors='ignore')
            else:
                pass
            increment = 5 if fnmatch.fnmatch(filename, '*xw_rutting*') else 5
            rut_df['event_start'] = range(0, len(rut_df) * increment, increment)
            rut_df['event_end'] = rut_df['event_start'] + increment
            rut_df['rut_chainage'] = rut_df.index * 25 // 5
            survey_code = filename.split('_')[2].split('.')[0]
            # rut_df.set_index('index', inplace=True)
            rut_df['survey_code'] = survey_code
            rut_df['rut_point_x'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[1]))
            rut_df['rut_point_y'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[0]))
            rut_df['rut_point_x'].fillna(0, inplace=True)
            rut_df['rut_point_y'].fillna(0, inplace=True)
        
            rut_df.rename(columns={'#Date':'Date', 'left rutting height': 'left_rutting', 'right rutting height': 'right_rutting', 'average height': 'avg_rutting'}, inplace=True)
            rut_df.drop(columns=['qgis_shape'], inplace=True)
            rutting_dataframes[filename] = rut_df

            # print(f"Updated {filename} into Rutting DataFrame.")

    print(f"✅ Finished processing: .CSV files.")
    return iri_dataframes, rutting_dataframes

เพิ่ม chainage event_str event_end ให้ iri/rut return แบบ concat ต่อกันทีละ RUN

In [3]:
def process_csv_files(path):
    all_iri_dataframes = []  # List to store all IRI dataframes
    all_rutting_dataframes = []  # List to store all Rutting dataframes

    for root, dirs, files in os.walk(path):
        # Find files
        iri_files = [f for f in files if f.endswith('.csv') and 'xw_iri_qgis' in f]
        rutting_files = [f for f in files if f.endswith('.csv') and 'xw_rutting' in f]

        # Process 'xw_iri_qgis' files
        for filename in iri_files:
            file_path = os.path.join(root, filename)
            iri_df = pd.read_csv(file_path, delimiter=';')
            iri_df.columns = iri_df.columns.str.strip()  # Clean column names
            survey_code = filename.split('_')[3].split('.')[0]  # Extract survey code
            iri_df['survey_code'] = survey_code
            iri_df['iri'] = (iri_df['iri left (m/km)'] + iri_df['iri right (m/km)']) / 2
            iri_df.drop(columns=['geometry'], errors='ignore', inplace=True)

            # Generate random values for iri_lane
            target_values = iri_df['iri']
            num_parts = 4
            tolerance = 0.3
            parts_list = generate_parts(target_values, num_parts, tolerance)

            # Expand DataFrame by repeating the rows
            iri_df = iri_df.loc[iri_df.index.repeat(num_parts)].reset_index(drop=True)
            iri_df['iri_lane'] = np.concatenate(parts_list)

            # Set initial event columns
            increment = 5
            # iri_df['iri_chainage'] = iri_df.index * increment
            iri_df['event_start'] = range(0, len(iri_df) * increment, increment)
            iri_df['event_end'] = iri_df['event_start'] + increment

            # Append the processed IRI DataFrame to the list
            all_iri_dataframes.append(iri_df)

        # Process 'xw_rutting' files
        for filename in rutting_files:
            file_path = os.path.join(root, filename)
            rut_df = pd.read_csv(file_path, delimiter=';')
            rut_df.columns = rut_df.columns.str.strip()
            if 'Unnamed: 5' in rut_df.columns:
                rut_df.drop(columns=['Unnamed: 5'], inplace=True, errors='ignore')
            else:
                pass
            increment = 5 if fnmatch.fnmatch(filename, '*xw_rutting*') else 5
            rut_df['event_start'] = range(0, len(rut_df) * increment, increment)
            rut_df['event_end'] = rut_df['event_start'] + increment
            rut_df['rut_chainage'] = rut_df.index * 25 // 5
            survey_code = filename.split('_')[2].split('.')[0]
            # rut_df.set_index('index', inplace=True)
            rut_df['survey_code'] = survey_code
            rut_df['rut_point_x'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[1]))
            rut_df['rut_point_y'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[0]))
            rut_df['rut_point_x'].fillna(0, inplace=True)
            rut_df['rut_point_y'].fillna(0, inplace=True)
        
            rut_df.rename(columns={'#Date':'Date', 'left rutting height': 'left_rutting', 'right rutting height': 'right_rutting', 'average height': 'avg_rutting'}, inplace=True)
            rut_df.drop(columns=['qgis_shape'], inplace=True)

            all_rutting_dataframes.append(rut_df)

    # Concatenate all IRI DataFrames into one
    if all_iri_dataframes:
        iri_dataframes = pd.concat(all_iri_dataframes, ignore_index=True)
    else:
        iri_dataframes = pd.DataFrame()  # Empty DataFrame if no IRI data found

    # Concatenate all Rutting DataFrames into one
    if all_rutting_dataframes:
        rutting_dataframes = pd.concat(all_rutting_dataframes, ignore_index=True)
    else:
        rutting_dataframes = pd.DataFrame()  # Empty DataFrame if no Rutting data found

    return iri_dataframes, rutting_dataframes

In [4]:
output_dir = r'D:\xenomatix\output'
iri_dataframes, rutting_dataframes = process_csv_files(output_dir)

In [6]:
iri_dataframes.to_csv('iri_test.csv')

In [7]:
rutting_dataframes.to_csv('rut_test.csv')

join rut/iri between event_str event_end and survey_code

In [8]:
def left_join_dataframes(df_rutting, df_iri):
    df_merged = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end', 'survey_code'], suffixes=('_rutting', '_iri'))
    
    print("Merged DataFrame columns:", df_merged.columns)

    result = df_merged[
        (df_merged['rut_chainage'] >= df_merged['event_start']) &
        (df_merged['rut_chainage'] < df_merged['event_end'])
    ]
    return result

joined_df = left_join_dataframes(rutting_dataframes, iri_dataframes)

Merged DataFrame columns: Index(['Date_rutting', 'left_rutting', 'right_rutting', 'avg_rutting',
       'event_start', 'event_end', 'rut_chainage', 'survey_code',
       'rut_point_x', 'rut_point_y', 'Date_iri', 'iri left (m/km)',
       'iri Std left (m/km)', 'iri right (m/km)', 'iri Std right (m/km)',
       'worst iri (m/km)', 'iri difference (m/km)', 'iri', 'iri_lane'],
      dtype='object')


เอา rut/iri ที่ process csv แบบ concat ที่ join between แล้ว มา join กับรูป ด้วย survey_code เพื่อหา frame_num, frame_num_ch

In [16]:
def left_join_dataframes(df_rutting, df_iri):
    df_merged = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end', 'survey_code'], suffixes=('_rutting', '_iri'))
    
    # print("Merged DataFrame columns:", df_merged.columns)

    result = df_merged[
        (df_merged['rut_chainage'] >= df_merged['event_start']) &
        (df_merged['rut_chainage'] < df_merged['event_end'])
    ]
    return result

def add_frame_num_to_joined_df(joined_df, derived_values, jpg_num):
    # Create new columns 'frame_num_ch' and 'frame_num' initialized with NaN
    joined_df['frame_ch'] = pd.NA
    joined_df['frame_num'] = pd.NA

    # Create a DataFrame to map derived values to frame numbers
    derived_to_frame_mapping = pd.DataFrame({
        'frame_num_ch': derived_values,
        'frame_num': jpg_num
    })

    # Additional logic to update frame_num_ch and frame_num based on derived_values
    for i, frame_num_ch in enumerate(derived_values):
        mask = (joined_df['event_start'] <= frame_num_ch) & (joined_df['event_end'] > frame_num_ch)
        joined_df.loc[mask, 'frame_num_ch'] = frame_num_ch
        joined_df.loc[mask, 'frame_num'] = jpg_num[i]
    return joined_df

def get_jpg_names_and_nums(directory):
    jpg_num_dict = {}
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)
            jpg_num = [int(files.split('-')[-1].split('.jpg')[0]) for files in files]
            jpg_num_dict = jpg_num
            
    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace(r'_(\d+)', lambda m: f"RUN{int(m.group(1)):02d}", regex=True)
    return jpg_files, frame_df, jpg_num

# Perform fainal data frame
def process_fainal_df(output_dir):
    jpg_files, frame_df, jpg_num = get_jpg_names_and_nums(output_dir)
    iri_dataframes, rutting_dataframes = process_csv_files(output_dir)

    joined_dataframes = []

    joined_df = left_join_dataframes(rutting_dataframes, iri_dataframes)
    
    grouped_df = joined_df.groupby('survey_code').agg(
        max_chainage=('rut_chainage', 'max'),
        min_chainage=('rut_chainage', 'min')
    ).reset_index()

    joined_df = pd.merge(joined_df, grouped_df, on='survey_code', how='left')
    joined_df = pd.merge(joined_df, frame_df, on='survey_code', how='left')
    
    max_event_start = joined_df['event_start'].max()
    
    derived_values = [round((max_event_start * num) / max(jpg_num)) for num in jpg_num]
    joined_df = add_frame_num_to_joined_df(joined_df, derived_values, jpg_num)

    joined_dataframes.append(joined_df)
                
    final_df = pd.concat(joined_dataframes, ignore_index=True)
    
    final_df = final_df.rename(columns={'rut_chainage':'chainage'})
    
    selected_columns = [
        'left_rutting', 'right_rutting', 'avg_rutting', 'event_start', 'event_end', 'survey_code',
        'rut_point_x', 'rut_point_y', 'Date', 'iri left (m/km)', 'iri right (m/km)', 'iri', 'iri_lane', 
        'chainage', 'max_chainage', 'min_chainage', 'pic_count', 'frame_num', 'frame_num_ch'
    ] 
    
    selected_columns = [col for col in selected_columns if col in final_df.columns]
    final_df = final_df[final_df['Date_rutting'].notnull()][selected_columns]
    # final_df = final_df[final_df['iri'].notnull()][selected_columns]
    
    return final_df

In [17]:
output_dir = r'D:\xenomatix\output'
final_df = process_fainal_df(output_dir)

# final_df.to_csv('joinjoin.csv')
final_df.tail(10)

,left_rutting,right_rutting,avg_rutting,event_start,event_end,survey_code,rut_point_x,rut_point_y,iri left (m/km),iri right (m/km),iri,iri_lane,chainage,max_chainage,min_chainage,pic_count,frame_num,frame_num_ch
2435,0.26,2.96,1.61,5990,5995,20240726RUN03,13.849395,100.444022,8.690157,10.062783,9.376470,9.387792,5990,6035,0,857,831,5993.0
2436,0.41,6.50,3.46,5995,6000,20240726RUN03,13.849415,100.443981,8.690157,10.062783,9.376470,9.644570,5995,6035,0,857,<NA>,NaN
2437,0.34,6.09,3.22,6000,6005,20240726RUN03,13.849436,100.443940,15.840459,16.280330,16.060394,15.790306,6000,6035,0,857,832,6000.0
2438,5.66,2.46,4.06,6005,6010,20240726RUN03,13.849459,100.443900,15.840459,16.280330,16.060394,15.503531,6005,6035,0,857,833,6007.0
2439,6.84,0.27,3.56,6010,6015,20240726RUN03,13.849488,100.443864,15.840459,16.280330,16.060394,17.579181,6010,6035,0,857,834,6014.0
2440,1.26,0.00,0.63,6015,6020,20240726RUN03,13.849523,100.443835,15.840459,16.280330,16.060394,15.255463,6015,6035,0,857,<NA>,NaN
2441,36.10,0.00,18.05,6020,6025,20240726RUN03,13.849565,100.443818,NaN,NaN,NaN,NaN,6020,6035,0,857,835,6021.0
2442,16.73,0.00,8.36,6025,6030,20240726RUN03,13.849609,100.443813,NaN,NaN,NaN,NaN,6025,6035,0,857,836,6029.0
2443,1.78,0.81,1.30,6030,6035,20240726RUN03,13.849654,100.443820,NaN,NaN,NaN,NaN,6030,6035,0,857,<NA>,NaN
2444,0.43,0.92,0.68,6035,6040,20240726RUN03,13.849698,100.443832,NaN,NaN,NaN,NaN,6035,6035,0,857,837,6036.0


เอา rut/iri ที่ process csv แบบ dic และที่ join แบบปกติ แล้ว มา join กับรูป ด้วย survey_code เพื่อหา frame_num, frame_num_ch

In [31]:
# Perform the left join on xw_rutting and xw_iri_qgis
def left_join_dataframes(df_rutting, df_iri):
    joined_df = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end'], suffixes=('_rutting', '_iri'))
    return joined_df

def add_frame_num_to_joined_df(joined_df, derived_values, jpg_num):
    # Create new columns 'frame_num_ch' and 'frame_num' initialized with NaN
    joined_df['frame_ch'] = pd.NA
    joined_df['frame_num'] = pd.NA

    # Create a DataFrame to map derived values to frame numbers
    derived_to_frame_mapping = pd.DataFrame({
        'frame_num_ch': derived_values,
        'frame_num': jpg_num
    })

    # Additional logic to update frame_num_ch and frame_num based on derived_values
    for i, frame_num_ch in enumerate(derived_values):
        mask = (joined_df['event_start'] <= frame_num_ch) & (joined_df['event_end'] > frame_num_ch)
        joined_df.loc[mask, 'frame_num_ch'] = frame_num_ch
        joined_df.loc[mask, 'frame_num'] = jpg_num[i]
    return joined_df

def get_jpg_names_and_nums(directory):
    jpg_num_dict = {}
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)
            jpg_num = [int(files.split('-')[-1].split('.jpg')[0]) for files in files]
            jpg_num_dict = jpg_num
            
    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace(r'_(\d+)', lambda m: f"RUN{int(m.group(1)):02d}", regex=True)
    return jpg_files, frame_df, jpg_num

# Perform fainal data frame
def process_fainal_df(output_dir):
    jpg_files, frame_df, jpg_num = get_jpg_names_and_nums(output_dir)
    iri_dataframes, rutting_dataframes = process_csv_files(output_dir)

    joined_dataframes = {}
    for rutting_file in rutting_dataframes:
        for iri_file in iri_dataframes:
            if 'xw_rutting' in rutting_file and 'xw_iri_qgis' in iri_file:
                # joined_df = left_join_dataframes(rutting_dataframes[rutting_file], iri_dataframes[iri_file])
                joined_df = left_join_dataframes(rutting_dataframes, iri_dataframes)
                
                grouped_df = joined_df.groupby('survey_code').agg(
                    max_chainage=('chainage', 'max'),
                    min_chainage=('chainage', 'min')
                ).reset_index()

                joined_df = pd.merge(joined_df, grouped_df, on='survey_code', how='left')
                joined_df = pd.merge(joined_df, frame_df, on='survey_code', how='left')
                
                max_event_start = joined_df['event_start'].max()
                
                derived_values = [round((max_event_start * num) / max(jpg_num)) for num in jpg_num]
                joined_df = add_frame_num_to_joined_df(joined_df, derived_values, jpg_num)

                # Store the joined DataFrame in the dictionary
                joined_dataframes[f"{rutting_file}_{iri_file}"] = joined_df
                
    final_df = pd.concat(joined_dataframes.values(), ignore_index=True)
    
    selected_columns = [
        'left_rutting', 'right_rutting', 'avg_rutting', 'event_start', 'event_end', 'survey_code',
        'rut_point_x', 'rut_point_y', 'Date', 'iri left (m/km)', 'iri right (m/km)', 'iri', 'iri_lane', 
        'chainage', 'max_chainage', 'min_chainage', 'pic_count', 'frame_num', 'frame_num_ch'
    ] 
    
    selected_columns = [col for col in selected_columns if col in final_df.columns]
    final_df = final_df[final_df['iri'].notnull()][selected_columns]
    return final_df

# edit 9/9

In [ ]:
output_dir = r'D:\xenomatix\output'
final_df = process_fainal_df(output_dir)

ใช้ final_df ที่ join between และ process csv แบบ concat เพื่อออก csv ใน function main

In [86]:
def find_csv_files(start_dir, prefix='log_'):
    csv_files = []
    for dirpath, dirnames, filenames in os.walk(start_dir):
        for filename in fnmatch.filter(filenames, f'{prefix}*.xlsx'):
            csv_files.append(os.path.join(dirpath, filename))
    return csv_files

def main(final_df):
    for survey_date in os.listdir(output_dir): # eg. base_dir = r"D:\xenomatixs"
        path = os.path.join(output_dir, survey_date, 'Output')
        mdb = os.path.join(output_dir, survey_date, 'Data')
        
        log_csv_files = find_csv_files(path)
        if log_csv_files:
            log_df = pd.read_excel(log_csv_files[0])
            log_df.rename(columns={'ผิว': 'event_name', 'link_id ระบบ': 'section_id'}, inplace=True)
            log_df.columns = log_df.columns.str.strip()

            folder_names = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
            for folder_name in folder_names:
                print(f"🔄 Processing folder: {folder_name}")
                
                # Perform the initial merge and filter rows where frame_num is between numb_start and numb_end
                merged_df = pd.merge(final_df, log_df, how='left', on=['survey_code'], suffixes=('_final_df', '_log_df'))
                print(merged_df.columns)
                merged_df = merged_df[(merged_df['frame_num'] >= merged_df['numb_start']) & 
                                    (merged_df['frame_num'] <= merged_df['numb_end'])]
                
                merged_df.to_csv('sss.csv')
                filtered_df = merged_df[merged_df['survey_code'] == folder_name]
                run_code = re.sub(r'RUN0*(\d+)', r'_\1', folder_name)

                def process_val(df):
                    df['chainage'] = df['chainage']
                    df['lon'] = df['rut_point_y']
                    df['lat'] = df['rut_point_x']
                    df['iri_right'] = df['iri right (m/km)']
                    df['iri_left'] = df['iri left (m/km)']
                    df['iri'] = df['iri']
                    df['iri_lane'] = df['iri_lane']
                    df['rutt_right'] = df['right_rutting']
                    df['rutt_left'] = df['left_rutting']
                    df['rutting'] = df['avg_rutting']
                    df['texture'] = 0
                    df['etd_texture'] = 0
                    df['event_name'] = df['event_name'].str.lower()
                    df['frame_number'] = df['frame_num']
                    df['file_name'] = df['survey_code'].str.replace(r'RUN0*(\d+)', r'_\1', regex=True)
                    df['run_code'] = df['file_name'].str.split('_').str[-1]

                    return df

                processed_val = process_val(merged_df)

                selected_columns_val = [
                    'chainage', 'lon', 'lat', 'iri_right', 'iri_left', 'iri', 'iri_lane', 'rutt_right', 'rutt_left', 
                    'rutting', 'texture', 'etd_texture', 'event_name', 'frame_number', 'file_name', 'run_code'
                ]

                selected_columns_val = [col for col in selected_columns_val if col in processed_val.columns]
                processed_val_filename = os.path.join(mdb, 'access_valuelaser.csv')
                processed_val[selected_columns_val].to_csv(os.path.join(processed_val_filename), index=False)
                
                def process_dis(df):
                    df['chainage_pic'] = df['chainage']
                    df['frame_number'] = df['frame_num']
                    df['event_name'] = df['event_name'].str.lower()
                    df['name_key'] = df['survey_code'].str.replace(r'RUN0*(\d+)', r'_\1', regex=True)
                    df['run_code'] = df['file_name'].str.split('_').str[-1]

                    return df

                processed_dis = process_dis(merged_df)

                selected_columns_dis = [
                    'chainage_pic', 'frame_number', 'event_name', 'name_key', 'run_code'
                ]

                selected_columns_dis = [col for col in selected_columns_dis if col in processed_dis.columns]
                processed_dis_filename = os.path.join(mdb, 'access_distress_pic.csv')
                processed_dis[selected_columns_dis].to_csv(os.path.join(processed_dis_filename), index=False)
                
                def process_key(df):
                    df['event_str'] = round(df['numb_start'] * (df['max_chainage'] / df['pic_count']))
                    df['event_end'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count']))
                    df['event_num'] = df['event_name'].str[0].str.lower()
                    df['event_type'] = 'pave type'
                    df['event_name'] = df['event_name'].str.lower()
                    df['link_id'] = df['linkid']
                    df['lane_no'] = df['linkid'].apply(lambda x: x[11:13])
                    df['survey_date'] = df['date']
                    df['lat_str'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('first')
                    df['lat_end'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('last')
                    df['lon_str'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('first')
                    df['lon_end'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('last')
                    df['name_key'] = df['survey_code'].str.replace(r'RUN0*(\d+)', r'_\1', regex=True)
                    df['run_code'] = df['name_key'].str.split('_').str[-1]
                    
                    return df

                processed_key = merged_df.groupby('survey_code', group_keys=False).apply(process_key).reset_index(drop=True)
                processed_key = processed_key.groupby(['linkid', 'survey_date']).first().reset_index()

                selected_columns_key = [
                    'event_str', 'event_end', 'event_num', 'event_type', 'event_name', 'link_id', 'section_id', 
                    'km_start', 'km_end', 'length', 'lane_no', 'survey_date', 'lat_str', 'lat_end', 'lon_str', 
                    'lon_end', 'name_key', 'run_code'
                ]

                selected_columns_key = [col for col in selected_columns_key if col in processed_key.columns]
                processed_key_filename = os.path.join(mdb, 'access_key.csv')
                processed_key[selected_columns_key].sort_values(by=['run_code', 'event_str', 'event_end'], ascending=[True, True, False]).to_csv(os.path.join(processed_key_filename), index=False)
# .csv
# .mdb 
                mdb_folder_path = os.path.join(mdb, run_code)
                # print(f'store in: {mdb_folder_path}')
                mdb_path = os.path.join(mdb_folder_path, f'{run_code}_edit.mdb')
                # print(f'this name: {mdb_path}')
            
                if not os.path.isdir(mdb):
                    print(f"⛔ Directory not found: {mdb}")
                    continue
                
                def mdb_video_process(df):
                    df['CHAINAGE'] = df['chainage']
                    df['LRP_OFFSET'] = df['chainage']
                    df['LRP_NUMBER'] = 0
                    df['FRAME'] = df['frame_num']
                    df['GPS_TIME'] = 0
                    df['X'] = df['rut_point_y']
                    df['Y'] = df['rut_point_x']
                    df['Z'] = 0
                    df['HEADING'] = 0
                    df['PITCH'] = 0
                    df['ROLL'] = 0

                    return df

                video_process = mdb_video_process(filtered_df)
                
                selected_mdb_video_process = [
                    'CHAINAGE', 'LRP_OFFSET', 'LRP_NUMBER', 'FRAME', 'GPS_TIME', 
                    'X', 'Y', 'Z', 'HEADING', 'PITCH', 'ROLL'
                ]

                selected_mdb_video_process = [col for col in selected_mdb_video_process if col in video_process.columns]
                mdb_video_process_filename = os.path.join(mdb_folder_path, f'Video_Processed_{run_code}_2.csv')
                video_process[selected_mdb_video_process].to_csv(mdb_video_process_filename, index=False)
                
                mdb_video_header = pd.DataFrame({
                    'CAMERA': [1, 2],
                    'NAME': ['ROW-0', 'PAVE-0'],
                    'DEVICE': ['XENO', 'XENO'],
                    'SERIAL': ['6394983', '6394984'],
                    'INTERVAL': [5, 2],
                    'WIDTH': [0, 0],
                    'HEIGHT': [0, 0],
                    'FRAME_RATE': [0, 0],
                    'FORMAT': ['422 YUV 8', 'Mono 8'],
                    'X_SCALE': [0, 0.5],
                    'Y_SCALE': [0, 0.5],
                    'DATA_FORMAT': [-1, -1],
                    'PROCESSING_METHOD': [-1, -1],
                    'ENABLE_MOBILE_MAPPING': [True, False],
                    'DISP_PITCH': [0, 0],
                    'DISP_ROLL': [0, 0],
                    'DISP_YAW': [0, 0],
                    'DISP_X': [0, 0],
                    'DISP_Y': [0, 0],
                    'DISP_Z': [0, 0],
                    'HFOV': [0, 0],
                    'VFOV': [0, 0]
                })
                
                mdb_video_header_filename = os.path.join(mdb_folder_path, f'Video_Header_{run_code}.csv')
                mdb_video_header.to_csv(mdb_video_header_filename, index=False)
                
                def mdb_survey_header(df):
                    current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    df['SURVEY_ID'] = run_code
                    df['SURVEY_FILE'] = run_code
                    df['SURVEY_DESC'] = None
                    df['SURVEY_DATE'] = current_datetime
                    df['VEHICLE'] = 'ISS'
                    df['OPERATOR'] = 'ISS'
                    df['USER_1_NAME'] = None
                    df['USER_1'] = None
                    df['USER_2_NAME'] = None
                    df['USER_2'] = None
                    df['USER_3_NAME'] = None
                    df['USER_3'] = None
                    df['LRP_FILE'] = f'LRP_{run_code}'
                    df['LRP_RESET'] = 'N'
                    df['LRP_START'] = 0
                    df['CHAIN_INIT'] = 0
                    df['CHAIN_START'] = 0
                    df['CHAIN_END'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count'])).max()
                    df['SECT_LEN'] = 0
                    df['DIR'] = 'I'
                    df['LANE'] = 1
                    df['DEVICES'] = 'GPS-Geo-DR,LP_V3-LWP,LP_V3-RWP,TPL,Video'
                    df['OTHERSIDE'] = True
                    df['VERSION'] = '2.7.3.4/2.7.3.4'
                    df['MEMO'] = None
                    df['LENGTH'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count'])).max()

                    df = df.astype({
                        'SURVEY_DATE': 'datetime64[ns]',
                        'LRP_START': 'int',
                        'CHAIN_INIT': 'int',
                        'CHAIN_START': 'int',
                        'CHAIN_END': 'int',
                        'SECT_LEN': 'int',
                        'LANE': 'int',
                        'OTHERSIDE': 'bool',
                        'LENGTH': 'int'
                    })
                    
                    return df

                survey_header = mdb_survey_header(filtered_df)
                survey_header = survey_header.groupby(['SURVEY_ID']).first().reset_index()
                
                selected_mdb_survey_header = [
                    'SURVEY_ID', 'SURVEY_FILE', 'SURVEY_DESC', 'SURVEY_DATE', 'VEHICLE', 'OPERATOR', 'USER_1_NAME', 'USER_1', 
                    'USER_2_NAME', 'USER_2', 'USER_3_NAME', 'USER_3', 'LRP_FILE', 'LRP_RESET', 'LRP_START', 'CHAIN_INIT', 
                    'CHAIN_START','CHAIN_END', 'SECT_LEN', 'DIR', 'LANE', 'DEVICES', 'OTHERSIDE', 'VERSION', 'MEMO', 'LENGTH'
                ]

                selected_mdb_survey_header = [col for col in selected_mdb_survey_header if col in survey_header.columns]
                mdb_survey_header_filename = os.path.join(mdb_folder_path, f'Survey_Header_{run_code}.csv')
                survey_header[selected_mdb_survey_header].to_csv(mdb_survey_header_filename, index=False)
                
                def mdb_KeyCode_Raw(df):
                    df['CHAINAGE_START'] = round(df['numb_start'] * (df['max_chainage'] / df['pic_count']))
                    df['CHAINAGE_END'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count']))
                    df['EVENT'] = df['event_name'].str[0].str.lower()
                    df['SWITCH_GROUP'] = 'pave type.'
                    df['EVENT_DESC'] = df['event_name'].str.lower()
                    df['LATITUDE_START'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('first')
                    df['LATITUDE_END'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('last')
                    df['LONGITUDE_START'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('first')
                    df['LONGITUDE_END'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('last')
                    df['link_id'] = df['linkid']
                    df['section_id'] = df['section_id']
                    df['km_start'] = df['km_start']
                    df['km_end'] = df['km_end']
                    df['length'] = df['length']
                    df['lane_no'] = df['linkid'].apply(lambda x: x[11:13])
                    df['survey_date'] = df['date']
                    
                    return df

                KeyCode_Raw = merged_df.groupby('survey_code', group_keys=False).apply(mdb_KeyCode_Raw).reset_index(drop=True)
                KeyCode_Raw = KeyCode_Raw.groupby(['linkid', 'survey_date']).first().reset_index()
                KeyCode_Raw = KeyCode_Raw[KeyCode_Raw['survey_code'] == folder_name]

                selected_mdb_KeyCode_Raw = [
                    'CHAINAGE_START', 'CHAINAGE_END', 'EVENT', 'SWITCH_GROUP', 'EVENT_DESC', 'LATITUDE_START', 'LATITUDE_END', 
                    'LONGITUDE_START', 'LONGITUDE_END', 'link_id', 'section_id', 'km_start', 'km_end', 'length', 'lane_no', 
                    'survey_date'
                ]

                selected_mdb_KeyCode_Raw = [col for col in selected_mdb_KeyCode_Raw if col in KeyCode_Raw.columns]
                mdb_KeyCode_Raw_filename = os.path.join(mdb_folder_path, f'KeyCode_Raw_{run_code}.csv')
                KeyCode_Raw[selected_mdb_KeyCode_Raw].sort_values(by=['lane_no', 'CHAINAGE_START', 'CHAINAGE_END'], ascending=[True, True, False]).to_csv(mdb_KeyCode_Raw_filename, index=False)    
# insert .mdb

In [ ]:
if __name__ == "__main__":
    try:
        main(final_df)
    except Exception as e:
        print(f"⛔ Error in the main block: {e}")

แก้การ join รูป ให้ chainage กับรูปจบเท่ากัน

แก้ คอลันม์ iri/rut เพิ่ม geom เพื่ม gen iri อาจจะใช้ code นี้ได้เพราะรูปมาครบ

In [ ]:
import pandas as pd
import os
import fnmatch

# Constants
DIRECTORY_PATH = r"D:\xenomatix\output\survey_data_20240726\Output\20240726RUN02"
JPG_DIRECTORY_PATH = r"D:\xenomatix\output\survey_data_20240726\PAVE\20240726_2\PAVE-0"

# Functions
def get_jpg_filenames(directory):
    jpg_dict = {}
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_num = [int(fname.split('-')[-1].split('.jpg')[0]) for fname in jpg_files]
    return jpg_num

def process_csv_file(file_path, increment):
    df = pd.read_csv(file_path, delimiter=';')
    
    df['event_start'] = 0
    df['event_end'] = increment
    
    event_start = 0
    event_end = increment
    
    for i in range(len(df)):
        df.at[i, 'event_start'] = event_start
        df.at[i, 'event_end'] = event_end
        event_start = event_end
        event_end += increment
    
    return df

def create_interval_rows(df, start_col='event_start', end_col='event_end', interval_size=5):
    new_rows = []
    
    for _, row in df.iterrows():
        start = row[start_col]
        end = row[end_col]
        
        while start < end:
            new_row = row.copy()
            new_row[start_col] = start
            new_row[end_col] = start + interval_size
            new_rows.append(new_row)
            start += interval_size
    
    return pd.DataFrame(new_rows)

def left_join_dataframes(df_rutting, df_iri):
    return pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end'], suffixes=('_rutting', '_iri'))

def add_frame_num_to_joined_df(joined_df, derived_values, frame_numbers):
    joined_df['frame_num_ch'] = pd.NA
    joined_df['frame_num'] = pd.NA
    
    derived_to_frame_mapping = pd.DataFrame({
        'frame_num_ch': derived_values,
        'frame_num': frame_numbers
    })
    
    for i, frame_num_ch in enumerate(derived_values):
        mask = (joined_df['event_start'] <= frame_num_ch) & (joined_df['event_end'] > frame_num_ch)
        joined_df.loc[mask, 'frame_num_ch'] = frame_num_ch
        joined_df.loc[mask, 'frame_num'] = frame_numbers[i]
    
    return joined_df

# Main
jpg_filenames = get_jpg_filenames(JPG_DIRECTORY_PATH)
frame_numbers = extract_numeric_from_filenames(jpg_filenames)

dataframes = {}

for filename in os.listdir(DIRECTORY_PATH):
    if filename.endswith(".csv"):
        file_path = os.path.join(DIRECTORY_PATH, filename)
        increment = 20 if fnmatch.fnmatch(filename, '*xw_iri_qgis*') else 5
        df = process_csv_file(file_path, increment)
        
        # Create new rows based on intervals
        new_df = create_interval_rows(df, interval_size=5)
        
        # Store the new DataFrame in the dictionary with the filename as the key
        dataframes[filename] = new_df

print("Processed and updated CSV files into DataFrames.")

# Perform the left join and store the result
joined_dataframes = {}

for rutting_file in dataframes:
    if fnmatch.fnmatch(rutting_file, '*xw_rutting*'):
        for iri_file in dataframes:
            if fnmatch.fnmatch(iri_file, '*xw_iri_qgis*'):
                joined_df = left_join_dataframes(dataframes[rutting_file], dataframes[iri_file])
                
                max_event_start = joined_df['event_start'].max()
                
                derived_values = [round((max_event_start * num) / max(frame_numbers)) for num in frame_numbers]
                print('max_event_end = ', max_event_start)
                print('frame_numbers = ', max(frame_numbers))
                print("Derived Values:", derived_values)
                print("Count of derived_values:", len(derived_values))
                print("Frame Numbers:", frame_numbers)
                print("Count of frame_numbers:", len(frame_numbers))
                
                joined_df = add_frame_num_to_joined_df(joined_df, derived_values, frame_numbers)
               
                joined_dataframes[f"{rutting_file}_{iri_file}"] = joined_df

print("Performed the left join and stored the result.")

output_dir = r'D:\xenomatix\output'
iri_dataframes, rutting_dataframes = process_csv_files(output_dir)

In [17]:
df

,Date,iri left (m/km),iri Std left (m/km),iri right (m/km),iri Std right (m/km),worst iri (m/km),iri difference (m/km),geometry,event_start,event_end
0,26/07/2024,3.865407,2.804227,3.482863,2.631087,3.865407,0.382544,LINESTRING(100.4438015813279 13.85304028353679...,0,20
1,26/07/2024,3.547611,2.251651,3.629724,2.853411,3.629724,0.082113,LINESTRING(100.4438338646643 13.85286229758905...,20,40
2,26/07/2024,3.097305,2.065051,3.022079,2.184983,3.097305,0.075226,LINESTRING(100.44385608110896 13.8526828659645...,40,60
3,26/07/2024,4.105208,3.417165,3.377205,2.298076,4.105208,0.728003,LINESTRING(100.44387122090556 13.8525027087672...,60,80
4,26/07/2024,2.856759,1.884442,2.991038,2.360168,2.991038,0.134279,LINESTRING(100.44388152103689 13.8523222191993...,80,100
...,...,...,...,...,...,...,...,...,...,...
303,26/07/2024,5.569864,4.323723,4.514422,3.205883,5.569864,1.055441,LINESTRING(100.48977019652955 13.8559590270556...,6060,6080
304,26/07/2024,6.403876,3.835795,5.234916,3.866694,6.403876,1.168960,LINESTRING(100.48993714618199 13.8560367764371...,6080,6100
305,26/07/2024,6.735491,4.039852,6.620786,6.458464,6.735491,0.114704,LINESTRING(100.49011085619544 13.8560987970925...,6100,6120
306,26/07/2024,4.782251,4.015040,5.930027,4.991943,5.930027,1.147776,LINESTRING(100.4902900220181 13.85614367654372...,6120,6140
